In [49]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import fdrcorrection, multipletests
import seaborn as sns

from matplotlib import rcParams, font_manager
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42

figuredir = '../../figures'
if not os.path.exists(figuredir):
    os.mkdir(figuredir)
    
assocdir = '../../data/tmp/HLA/Association'
if not os.path.exists(assocdir):
    os.mkdir(assocdir)

In [24]:
hlabase = '../../data/tmp/HLA/Stanford/Predictions/Omni25AffyMerged.{g}.Nclass_7.Flank_500K.4dig.Model.20220206.AllProbabilities.tsv'
genes = ['A', 'B', 'C', 'DRB1', 'DQB1', 'DQA1', 'DPB1', 'DPA1']
all_allele_data = []
import sys
for g in genes:
    hlafn = hlabase.format(g=g)
    hladat = pd.read_csv(hlafn, sep='\t')
    x1 = hladat.groupby(lambda x: x.split('/')[0], axis=0).sum()
    x2 = hladat.groupby(lambda x: x.split('/')[1], axis=0).sum()
    all_alleles = sorted(set(x1.index).union(x2.index))
    dosage = x1.loc[all_alleles, :].replace(np.nan, 0) + x2.loc[all_alleles, :].replace(np.nan, 0)
    dosage.index = [g+'*'+x for x in dosage.index]
    all_allele_data.append(dosage.T)
    
all_allele_data = pd.concat(all_allele_data, axis=1)

In [25]:
AF = all_allele_data.sum(axis=0) / (all_allele_data.shape[0]*2)
AF.sort_values().head()

DQA1*03:02    0.000010
DPB1*05:01    0.000015
C*12:02       0.000019
DPB1*02:02    0.000022
A*25:01       0.000031
dtype: float64

In [26]:
dosagefn = os.path.join(assocdir, 'AllGene_Dosage.tsv')
all_allele_data.to_csv(dosagefn, sep='\t')

In [27]:
all_allele_data_maf01 = all_allele_data.loc[:, (AF>.01) & (AF<.99)]

In [28]:
dosagefn = os.path.join(assocdir, 'AllGene_Dosage_Maf_01.tsv')
all_allele_data_maf01.to_csv(dosagefn, sep='\t')

In [29]:
phenotypes = ['MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl', 'MergeEpoch_SL_AgORSeqPos_Vs_PopControl']
covarmap = {'MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl': ['SEX', 'PCs'],
 'MergeEpoch_SL_AgORSeqPos_Vs_PopControl': ['SEX','PCs']}

tmpdir = '../../data/tmp/'
gwasdir = os.path.join(tmpdir, 'GWAS')
resdir = os.path.join(gwasdir, 'results_noH3covar_20220207')

In [30]:

results = {}
AF_thresh = .01
AF_both = {}
for p in phenotypes:
    pdir = '{base}/{p}'.format(base=resdir, p=p)
    phenoresdir = os.path.join(pdir, 'results')
    rawgwasdir = os.path.join(phenoresdir, 'raw_gwas')

    covarstr = ''.join(covarmap[p])
    nulloutfile = os.path.join(rawgwasdir, "gmmat.%s.%s.nullmodel.rds" % (covarstr, p))
    phenofn = os.path.join(pdir, 'Covar_Pheno_UnrelatedPCs.txt')
    relfn = os.path.join(pdir, 'rel/OmniH3Merged_PrePost2016.20210817.FiltSLOmni5.Geno_1e-1.filtBatchvars20220107.Deduped.{p}.maf_1e-2.hwe_1e-6.geno_5e-2.rel.withids.tsv').format(p=p)
    pheno = pd.read_csv(phenofn, sep='\t')
    rel = pd.read_csv(relfn, sep='\t', index_col=0)
    display((pheno['IID']==rel.index).value_counts())
    genot = all_allele_data.loc[rel.index,:].T
    AF = genot.sum(axis=1) / (genot.shape[1]*2)
    AF_both[p] = AF
    genot = genot.loc[(AF>AF_thresh) & (AF < (1- AF_thresh)), :]    
    samps = list(rel.index)
    genot['SNP'] = genot.index
    genot['REF'] = 'A'
    genot['ALT'] = 'T'
    genot = genot.loc[:, ['SNP', 'REF', 'ALT'] + samps]
    genotfn = os.path.join(assocdir, 'AllGene_Dosage_Maf_01.{p}.tsv'.format(p=p))
    genot.to_csv(genotfn, sep='\t', index=False)
    outfn = os.path.join(assocdir, 'AllGene_Dosage_Maf_01.{p}.results.tsv'.format(p=p))    
    cmd = 'Rscript ../../code/gwas_analysis/run_GMMAT_premade_model_text_dosage.R %s %s %s' % (nulloutfile, genotfn, outfn)
    print(cmd)
    !{cmd}
    results[p] = pd.read_csv(outfn, sep='\t')
    results[p].index = results[p]['SNP']
    _,q,_,_ = multipletests(results[p]['PVAL'].values, method='fdr_bh')
    results[p]['QVAL'] = q

True    1598
Name: IID, dtype: int64

Rscript ../../code/gwas_analysis/run_GMMAT_premade_model_text_dosage.R ../../data/tmp/GWAS/results_noH3covar_20220207/MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl/results/raw_gwas/gmmat.SEXPCs.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.nullmodel.rds ../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.tsv ../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.results.tsv
Warning message:
In glmm.score(model0, infile = genotypefn, outfile = outfn, is.dosage = FALSE,  :
  Argument select is unspecified... Assuming the order of individuals in infile matches unique id_include in obj...


True    921
Name: IID, dtype: int64

Rscript ../../code/gwas_analysis/run_GMMAT_premade_model_text_dosage.R ../../data/tmp/GWAS/results_noH3covar_20220207/MergeEpoch_SL_AgORSeqPos_Vs_PopControl/results/raw_gwas/gmmat.SEXPCs.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.nullmodel.rds ../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.tsv ../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.results.tsv
Warning message:
In glmm.score(model0, infile = genotypefn, outfile = outfn, is.dosage = FALSE,  :
  Argument select is unspecified... Assuming the order of individuals in infile matches unique id_include in obj...


In [31]:
for p in phenotypes:
    print(p)
    display(results[p].sort_values(by='PVAL').head(5))
    print('\n\n')

MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl


,SNP,N,AF,SCORE,VAR,PVAL,QVAL
SNP,,,,,,,
C*14:02,C*14:02,1598,0.018615,7.05560,6.09635,0.004269,0.381949
B*15:16,B*15:16,1598,0.026584,8.06405,8.94377,0.007008,0.381949
B*52:01,B*52:01,1598,0.014350,-4.81087,4.00562,0.016228,0.584849
A*02:01,A*02:01,1598,0.096311,-11.67760,29.38770,0.031230,0.584849
DQA1*05:01,DQA1*05:01,1598,0.051717,-8.47301,17.27290,0.041479,0.584849





MergeEpoch_SL_AgORSeqPos_Vs_PopControl


,SNP,N,AF,SCORE,VAR,PVAL,QVAL
SNP,,,,,,,
DPA1*02:02,DPA1*02:02,921,0.274253,-14.32660,26.92320,0.005761,0.378817
DQB1*05:01,DQB1*05:01,921,0.123551,11.21200,18.08820,0.008383,0.378817
DPA1*02:01,DPA1*02:01,921,0.445456,15.57400,38.93680,0.012566,0.378817
C*07:01,C*07:01,921,0.045904,6.47016,6.99465,0.014428,0.378817
DRB1*15:03,DRB1*15:03,921,0.022577,4.52815,3.58392,0.016762,0.378817


In [9]:
in1 = '../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.results.tsv'
in2 = '../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.results.tsv'
outfn = '../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.NG_SL_Susceptibility_MetaAnalysis.results.tsv'

In [ ]:
for fn in [in1, in2]:
    dat = pd.read_csv(fn, sep='\t')
    ## The run_GMMAT_meta.R script expects a REF and ALT column
    dat['REF'] = 'A'
    dat['ALT'] = 'T'
    dat = dat[['SNP', 'REF', 'ALT', 'N', 'AF', 'SCORE', 'VAR', 'PVAL']]
    dat.to_csv(fn.replace('.tsv', '.fixed.tsv'), sep='\t', index=False)

Rcmd = 'Rscript ../../code/gwas_analysis/run_GMMAT_meta.R %s %s %s' % (in1.replace('.tsv', '.fixed.tsv'), in2.replace('.tsv', '.fixed.tsv'), outfn)
print(Rcmd)
!{Rcmd}

cmd = 'rm ../../data/tmp/HLA/Association/*.fixed.tsv'
!{cmd}

In [10]:
meta = pd.read_csv(outfn, sep='\t')
meta.index = meta['SNP']
meta_both = meta.loc[meta['N']==2519, :]
_,q,_,_ = multipletests(meta_both['PVAL'].values, method='fdr_bh')
meta_both['QVAL'] = q
meta_both.sort_values(by='PVAL').head(10)

_,q,_,_ = multipletests(meta['PVAL'].values, method='fdr_bh')
meta['QVAL'] = q

/opt/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
Rcmdbase = 'Rscript --vanilla ../../code/gwas_analysis/run_GMMAT_fromDosage_text_Wald.R %s %s %s %s %s %d'
ncores = 15
for p in phenotypes:
    pdir = '{base}/{p}'.format(base=resdir, p=p)
    phenoresdir = os.path.join(pdir, 'results')
    rawgwasdir = os.path.join(phenoresdir, 'raw_gwas')
    formula ='"{p} ~ SEX + PC1 + PC2 + PC3 + PC4 + PC5 + PC6"'.format(p=p)
    covarstr = ''.join(covarmap[p])
    nulloutfile = os.path.join(rawgwasdir, "gmmat.%s.%s.nullmodel.rds" % (covarstr, p))
    phenofn = os.path.join(pdir, 'Covar_Pheno_UnrelatedPCs.txt')
    relfn = os.path.join(pdir, 'rel/OmniH3Merged_PrePost2016.20210817.FiltSLOmni5.Geno_1e-1.filtBatchvars20220107.Deduped.{p}.maf_1e-2.hwe_1e-6.geno_5e-2.rel.withids.tsv').format(p=p)
    pheno = pd.read_csv(phenofn, sep='\t')
    genotfn = os.path.join(assocdir, 'AllGene_Dosage_Maf_01.{p}.tsv'.format(p=p))
    outfn = os.path.join(assocdir, 'AllGene_Dosage_Maf_01.{p}.results.wald.tsv'.format(p=p))    

    Rcmd = Rcmdbase % (phenofn, genotfn, relfn, formula, outfn, ncores)
    print(Rcmd)
    #!{Rcmd}
    wald_dat = pd.read_csv(outfn, sep='\t')
    wald_dat.index = wald_dat['SNP']
    results[p]['OR'] = wald_dat.loc[results[p].index, 'BETA'].apply(np.exp)

Rscript --vanilla ../../code/gwas_analysis/run_GMMAT_fromDosage_text_Wald.R ../../data/tmp/GWAS/results_noH3covar_20220207/MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl/Covar_Pheno_UnrelatedPCs.txt ../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.tsv ../../data/tmp/GWAS/results_noH3covar_20220207/MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl/rel/OmniH3Merged_PrePost2016.20210817.FiltSLOmni5.Geno_1e-1.filtBatchvars20220107.Deduped.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.maf_1e-2.hwe_1e-6.geno_5e-2.rel.withids.tsv "MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl ~ SEX + PC1 + PC2 + PC3 + PC4 + PC5 + PC6" ../../data/tmp/HLA/Association/AllGene_Dosage_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.results.wald.tsv 15
Rscript --vanilla ../../code/gwas_analysis/run_GMMAT_fromDosage_text_Wald.R ../../data/tmp/GWAS/results_noH3covar_20220207/MergeEpoch_SL_AgORSeqP

In [12]:
for p in phenotypes:
    display(results[p].sort_values(by='PVAL').head())

,SNP,N,AF,SCORE,VAR,PVAL,QVAL,OR
SNP,,,,,,,,
C*14:02,C*14:02,1598,0.018615,7.05560,6.09635,0.004269,0.381949,2.793635
B*15:16,B*15:16,1598,0.026584,8.06405,8.94377,0.007008,0.381949,2.271244
B*52:01,B*52:01,1598,0.014350,-4.81087,4.00562,0.016228,0.584849,0.262185
A*02:01,A*02:01,1598,0.096311,-11.67760,29.38770,0.031230,0.584849,0.660209
DQA1*05:01,DQA1*05:01,1598,0.051717,-8.47301,17.27290,0.041479,0.584849,0.592874


,SNP,N,AF,SCORE,VAR,PVAL,QVAL,OR
SNP,,,,,,,,
DPA1*02:02,DPA1*02:02,921,0.274253,-14.32660,26.92320,0.005761,0.378817,0.565302
DQB1*05:01,DQB1*05:01,921,0.123551,11.21200,18.08820,0.008383,0.378817,1.760462
DPA1*02:01,DPA1*02:01,921,0.445456,15.57400,38.93680,0.012566,0.378817,1.494460
C*07:01,C*07:01,921,0.045904,6.47016,6.99465,0.014428,0.378817,2.180002
DRB1*15:03,DRB1*15:03,921,0.022577,4.52815,3.58392,0.016762,0.378817,2.681294


In [13]:
hits = []
pthresh = .05
for p in phenotypes:
    hits += list(results[p].loc[results[p]['PVAL']< pthresh, 'SNP'])


#hits = meta_both.loc[meta_both['PVAL']<.05, 'SNP']
    
ngdat = results['MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl'].loc[hits, ['AF', 'OR', 'PVAL']]
sldat = results['MergeEpoch_SL_AgORSeqPos_Vs_PopControl'].loc[hits, ['AF', 'OR', 'PVAL']]


for v in ['DRB1*08:06', 'B*78:01']:
    ngdat.loc[v, 'AF'] = AF_both['MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl'].loc[v]


ngdat.columns = [('NG', x) for x in ngdat.columns]
sldat.columns = [('SL', x) for x in sldat.columns]
zmeta = meta.loc[hits, ['PVAL', 'QVAL']]
zmeta.columns = [('Meta', x) for x in zmeta.columns]

comb = pd.concat([ngdat, sldat, zmeta], axis=1)
comb.columns = pd.MultiIndex.from_tuples(comb.columns)
comb.sort_values(by=('Meta', 'PVAL'))

/opt/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:1404: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


NG                            SL                      \
                   AF        OR      PVAL        AF        OR      PVAL   
SNP                                                                       
B*15:16      0.026584  2.271244  0.007008  0.018077  1.482257  0.458528   
C*14:02      0.018615  2.793635  0.004269  0.019571  1.398048  0.515944   
DRB1*15:03   0.243576  1.266862  0.055630  0.022577  2.681294  0.016762   
DPA1*02:02   0.180110  0.904495  0.482959  0.274253  0.565302  0.005761   
B*52:01      0.014350  0.262185  0.016228  0.010389  0.999957  0.999707   
B*78:01      0.002675       NaN       NaN  0.029817  2.262743  0.033891   
DRB1*08:06   0.000968       NaN       NaN  0.022056  0.135602  0.041165   
DPA1*02:01   0.354411  1.071581  0.520416  0.445456  1.494460  0.012566   
C*03:02      0.026070  0.797934  0.527066  0.045552  0.402719  0.049165   
DQB1*02:01   0.054898  0.603755  0.047375  0.057394  0.995807  0.990269   
DRB1*03:01   0.052193  0.602298  0.046988  0.054675  1.043818  0.904740   
DQA1*05:01   0.051717  0.592874  0.041479  0.058139  1.135128  0.704067   
DPB1*104:01  0.035886  0.554805  0.047785  0.013688  1.985782  0.211152   
DQB1*05:01   0.144763  0.955528  0.746068  0.123551  1.760462  0.008383   
C*07:01      0.058280  0.943455  0.793514  0.045904  2.180002  0.014428   
A*02:01      0.096311  0.660209  0.031230  0.148679  1.570313  0.090363   

                 Meta            
                 PVAL      QVAL  
SNP                              
B*15:16      0.006684  0.443502  
C*14:02      0.006876  0.443502  
DRB1*15:03   0.015897  0.669291  
DPA1*02:02   0.027561  0.669291  
B*52:01      0.027666  0.669291  
B*78:01      0.033891  0.669291  
DRB1*08:06   0.041165  0.669291  
DPA1*02:01   0.054253  0.669291  
C*03:02      0.070048  0.669291  
DQB1*02:01   0.101615  0.669291  
DRB1*03:01   0.111988  0.669291  
DQA1*05:01   0.146163  0.724313  
DPB1*104:01  0.173401  0.724313  
DQB1*05:01   0.285913  0.773177  
C*07:01      0.313120  0.773177  
A*02:01      0.383552  0.773177

In [14]:
pd.concat([
    results['MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl'].loc['B*52:01', :],
    results['MergeEpoch_SL_AgORSeqPos_Vs_PopControl'].loc['B*52:01', :],
    meta.loc['B*52:01', :].drop(['A1', 'A2'])
], axis=1).T

/opt/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,AF,N,OR,PVAL,QVAL,SCORE,SNP,VAR
B*52:01,0.0143497,1598,0.262185,0.0162281,0.584849,-4.81087,B*52:01,4.00562
B*52:01,0.0103885,921,0.999957,0.999707,0.999707,-0.000321926,B*52:01,0.768302
B*52:01,0.0129014,2519,NaN,0.0276661,0.669291,-4.81119,B*52:01,4.77392


In [15]:
meta.loc['B*52:01', :]

SNP        B*52:01
A1            True
A2               A
N             2519
AF       0.0129014
SCORE     -4.81119
VAR        4.77392
PVAL     0.0276661
QVAL      0.669291
Name: B*52:01, dtype: object

## Repeat with 2 digit

In [84]:
hlabase = '../../data/tmp/HLA/Stanford/Predictions/Omni25AffyMerged.{g}.Nclass_7.Flank_500K.4dig.Model.20220206.AllProbabilities.tsv'
genes = ['A', 'B', 'C', 'DRB1', 'DQB1', 'DQA1', 'DPB1', 'DPA1']
all_allele_data = []
import sys
for g in genes:
    hlafn = hlabase.format(g=g)
    hladat = pd.read_csv(hlafn, sep='\t')
    x1 = hladat.groupby(lambda x: x.split('/')[0].split(':')[0], axis=0).sum()
    x2 = hladat.groupby(lambda x: x.split('/')[1].split(':')[0], axis=0).sum()
    all_alleles = sorted(set(x1.index).union(x2.index))
    dosage = x1.loc[all_alleles, :].replace(np.nan, 0) + x2.loc[all_alleles, :].replace(np.nan, 0)
    dosage.index = [g+'*'+x for x in dosage.index]
    all_allele_data.append(dosage.T)
    
all_allele_data = pd.concat(all_allele_data, axis=1)
all_allele_data.head()

,A*01,A*02,A*03,A*11,A*23,A*24,A*25,A*26,A*29,A*30,...,DPB1*39,DPB1*40,DPB1*417,DPB1*461,DPB1*463,DPB1*85,DPA1*01,DPA1*02,DPA1*03,DPA1*04
WG0284565-DNA_A02_G-5231,8.980594e-10,3.592218e-04,5.966318e-05,9.884059e-28,0.001356,6.517996e-26,1.178850e-11,2.337478e-06,2.469260e-18,0.998225,...,1.498780e-28,0.001438,5.414226e-10,7.268555e-07,7.144158e-07,8.437959e-09,0.001149,1.998851,8.745754e-09,1.303046e-08
WG0284565-DNA_A03_G-5686,8.980982e-10,1.000359e+00,5.966575e-05,1.141340e-27,0.002020,6.890107e-30,3.488029e-32,4.674195e-19,2.559077e-18,0.997561,...,1.110960e-48,0.001508,1.893355e-09,4.785941e-04,7.322280e-04,3.749667e-04,0.004427,1.992840,2.733096e-03,1.604225e-10
WG0284565-DNA_A04_G-5690,5.607632e-10,2.242875e-04,5.939966e-05,6.542605e-19,0.996537,1.654490e-18,3.051315e-40,2.073262e-14,9.195031e-19,0.999054,...,2.338300e-02,0.976616,7.352054e-27,3.223394e-23,1.032338e-26,1.520536e-12,0.134395,0.949052,9.165531e-01,3.764090e-17
WG0284565-DNA_A05_G-5135,8.900894e-10,2.045765e-02,5.954924e-05,1.149885e-27,0.002043,1.019281e-25,3.309208e-39,4.663327e-19,2.469627e-18,0.997541,...,1.189321e-17,0.001541,9.483453e-10,1.336075e-04,1.336194e-04,7.503476e-09,1.006336,0.991862,1.801766e-03,1.591168e-12
WG0284565-DNA_A06_G-5045,3.665769e-18,2.174426e-16,1.148462e-09,7.008001e-20,0.000017,1.155316e-09,3.674382e-28,2.300854e-27,1.000000e+00,0.000001,...,1.247548e-01,0.870925,1.335608e-31,1.175743e-42,5.877356e-32,1.000000e+00,0.000007,0.999996,9.999962e-01,1.419585e-21


In [85]:
dosagefn = os.path.join(assocdir, 'AllGene_Dosage_2dig.tsv')
all_allele_data.to_csv(dosagefn, sep='\t')

In [86]:
AF = all_allele_data.sum(axis=0) / (all_allele_data.shape[0]*2)
AF.sort_values().head()

DPB1*05     0.000015
A*25        0.000031
DPB1*20     0.000060
DPB1*106    0.000097
A*11        0.000203
dtype: float64

In [87]:
all_allele_data_maf01 = all_allele_data.loc[:, (AF>.01) & (AF<.99)]

In [88]:
dosagefn = os.path.join(assocdir, 'AllGene_Dosage_2dig_Maf_01.tsv')
all_allele_data_maf01.to_csv(dosagefn, sep='\t')

In [89]:
phenotypes = ['MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl', 'MergeEpoch_SL_AgORSeqPos_Vs_PopControl']
covarmap = {'MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl': ['SEX', 'PCs'],
 'MergeEpoch_SL_AgORSeqPos_Vs_PopControl': ['SEX','PCs']}

tmpdir = '../../data/tmp/'
gwasdir = os.path.join(tmpdir, 'GWAS')
resdir = os.path.join(gwasdir, 'results_noH3covar_20220207')

In [91]:
results = {}
AF_thresh = .01
AF_both = {}
for p in phenotypes:
    pdir = '{base}/{p}'.format(base=resdir, p=p)
    phenoresdir = os.path.join(pdir, 'results')
    rawgwasdir = os.path.join(phenoresdir, 'raw_gwas')

    covarstr = ''.join(covarmap[p])
    nulloutfile = os.path.join(rawgwasdir, "gmmat.%s.%s.nullmodel.rds" % (covarstr, p))
    phenofn = os.path.join(pdir, 'Covar_Pheno_UnrelatedPCs.txt')
    relfn = os.path.join(pdir, 'rel/OmniH3Merged_PrePost2016.20210817.FiltSLOmni5.Geno_1e-1.filtBatchvars20220107.Deduped.{p}.maf_1e-2.hwe_1e-6.geno_5e-2.rel.withids.tsv').format(p=p)
    pheno = pd.read_csv(phenofn, sep='\t')
    rel = pd.read_csv(relfn, sep='\t', index_col=0)
    display((pheno['IID']==rel.index).value_counts())
    genot = all_allele_data.loc[rel.index,:].T
    AF = genot.sum(axis=1) / (genot.shape[1]*2)
    AF_both[p] = AF
    genot = genot.loc[(AF>AF_thresh) & (AF < (1- AF_thresh)), :]    
    samps = list(rel.index)
    genot['SNP'] = genot.index
    genot['REF'] = 'A'
    genot['ALT'] = 'T'
    genot = genot.loc[:, ['SNP', 'REF', 'ALT'] + samps]
    genotfn = os.path.join(assocdir, 'AllGene_Dosage_2dig_Maf_01.{p}.tsv'.format(p=p))
    genot.to_csv(genotfn, sep='\t', index=False)
    outfn = os.path.join(assocdir, 'AllGene_Dosage_2dig_Maf_01.{p}.results.tsv'.format(p=p))    
    cmd = 'Rscript ../../code/gwas_analysis/run_GMMAT_premade_model_text_dosage.R %s %s %s' % (nulloutfile, genotfn, outfn)
    print(cmd)
    !{cmd}
    results[p] = pd.read_csv(outfn, sep='\t')
    results[p].index = results[p]['SNP']
    _,q,_,_ = multipletests(results[p]['PVAL'].values, method='fdr_bh')
    results[p]['QVAL'] = q

True    1598
Name: IID, dtype: int64

Rscript ../../code/gwas_analysis/run_GMMAT_premade_model_text_dosage.R ../../data/tmp/GWAS/results_noH3covar_20220207/MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl/results/raw_gwas/gmmat.SEXPCs.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.nullmodel.rds ../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.tsv ../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.results.tsv
Warning message:
In glmm.score(model0, infile = genotypefn, outfile = outfn, is.dosage = FALSE,  :
  Argument select is unspecified... Assuming the order of individuals in infile matches unique id_include in obj...


True    921
Name: IID, dtype: int64

Rscript ../../code/gwas_analysis/run_GMMAT_premade_model_text_dosage.R ../../data/tmp/GWAS/results_noH3covar_20220207/MergeEpoch_SL_AgORSeqPos_Vs_PopControl/results/raw_gwas/gmmat.SEXPCs.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.nullmodel.rds ../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.tsv ../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.results.tsv
Warning message:
In glmm.score(model0, infile = genotypefn, outfile = outfn, is.dosage = FALSE,  :
  Argument select is unspecified... Assuming the order of individuals in infile matches unique id_include in obj...


In [92]:
for p in phenotypes:
    display(results[p].sort_values(by='PVAL').head())

,SNP,N,AF,SCORE,VAR,PVAL,QVAL
SNP,,,,,,,
C*14,C*14,1598,0.022431,7.69268,7.50715,0.004991,0.379295
B*52,B*52,1598,0.014350,-4.81087,4.00562,0.016228,0.495068
DQB1*02,DQB1*02,1598,0.221116,-17.07740,57.58010,0.024415,0.495068
A*02,A*02,1598,0.155092,-15.03510,45.64630,0.026056,0.495068
DPB1*104,DPB1*104,1598,0.035886,-7.20772,13.26120,0.047785,0.498404


,SNP,N,AF,SCORE,VAR,PVAL,QVAL
SNP,,,,,,,
DQB1*05,DQB1*05,921,0.168250,13.57760,23.66380,0.005253,0.400038
A*66,A*66,921,0.019237,-4.47821,3.08643,0.010802,0.400038
DRB1*15,DRB1*15,921,0.022852,4.60287,3.59448,0.015191,0.400038
B*78,B*78,921,0.029817,4.55013,4.60062,0.033891,0.558163
B*27,B*27,921,0.027429,-3.60119,3.37605,0.050004,0.558163


In [93]:
in1 = '../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.results.tsv'
in2 = '../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.results.tsv'
outfn = '../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.NG_SL_Susceptibility_MetaAnalysis.results.tsv'

In [94]:
for fn in [in1, in2]:
    dat = pd.read_csv(fn, sep='\t')
    ## The run_GMMAT_meta.R script expects a REF and ALT column
    dat['REF'] = 'A'
    dat['ALT'] = 'T'
    dat = dat[['SNP', 'REF', 'ALT', 'N', 'AF', 'SCORE', 'VAR', 'PVAL']]
    dat.to_csv(fn.replace('.tsv', '.fixed.tsv'), sep='\t', index=False)

Rcmd = 'Rscript ../../code/gwas_analysis/run_GMMAT_meta.R %s %s %s' % (in1.replace('.tsv', '.fixed.tsv'), in2.replace('.tsv', '.fixed.tsv'), outfn)
print(Rcmd)
!{Rcmd}

cmd = 'rm ../../data/tmp/HLA/Association/*.fixed.tsv'
!{cmd}

Rscript ../../code/gwas_analysis/run_GMMAT_meta.R ../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_NG_ExactLVMatchqPCR2xPosOrSeqPos_Vs_PopControl.results.fixed.tsv ../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.MergeEpoch_SL_AgORSeqPos_Vs_PopControl.results.fixed.tsv ../../data/tmp/HLA/Association/AllGene_Dosage_2dig_Maf_01.NG_SL_Susceptibility_MetaAnalysis.results.tsv


In [95]:
meta = pd.read_csv(outfn, sep='\t')
meta.index = meta['SNP']
_,q,_,_ = multipletests(meta['PVAL'].values, method='fdr_bh')
meta['QVAL'] = q

In [96]:
meta.sort_values(by='PVAL').head(5)

,SNP,A1,A2,N,AF,SCORE,VAR,PVAL,QVAL
SNP,,,,,,,,,
C*14,C*14,True,A,2519,0.021392,8.829350,10.618120,0.006736,0.547415
DRB1*15,DRB1*15,True,A,2519,0.164285,19.869970,68.849480,0.016635,0.547415
B*52,B*52,True,A,2519,0.012901,-4.811192,4.773922,0.027666,0.547415
B*78,B*78,True,A,921,0.029817,4.550130,4.600620,0.033891,0.547415
DQA1*01,DQA1*01,True,A,2519,0.499743,21.819600,110.190300,0.037652,0.547415
